**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!sudo pip install scikit-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json


from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten


Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

>  ```epsilon``` controls the capacity of the agent to explore more actions/states. We explore the environment with probability ```epsilon```, This parameter has to take in consideration a certain trade off , between exploration and exploitation, the more this parameter is big , the more we explore and the less we exploit already visited actions/states. In the other hand, if the parameter is small, we exploit more the already 'konwn' actions/states, and we don't explore more actions/states that can be more 'useful' for the agent. So the function act permits to exploit this epsilone greedy strategy where in a state s wether the agent chose a random action, or we chose an action already learned by the agent in that given state s.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=45 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> **Board** corresponds to the matrix representing the positions where there is a bonus or a malus or nothing (because it's the sum of the sum of bonus and malus ).  The first state of the rat in the **board** is set to 0 which means that the rat will not start neither with a bonus or a malus. So **board** represents the reward distribution over the **NxN ** grid.

> **Position** indicates the borders (the states that are not allowed to the agent and which are assigned to -1), the allowed states which are assigned to 0, and the current position of the agent which is assigent to 1. And that in each step of the game.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action, size=1)[0]
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
        
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/13.0. Average score (-5.0)
Win/lose count 6.5/16.0. Average score (-7.25)
Win/lose count 11.5/13.0. Average score (-5.333333333333333)
Win/lose count 8.0/9.0. Average score (-4.25)
Win/lose count 6.5/18.0. Average score (-5.7)
Win/lose count 9.5/10.0. Average score (-4.833333333333333)
Win/lose count 15.5/15.0. Average score (-4.071428571428571)
Win/lose count 12.5/11.0. Average score (-3.375)
Win/lose count 5.0/9.0. Average score (-3.4444444444444446)
Win/lose count 12.5/15.0. Average score (-3.35)
Win/lose count 4.5/13.0. Average score (-3.8181818181818183)
Win/lose count 10.5/13.0. Average score (-3.7083333333333335)
Win/lose count 6.0/15.0. Average score (-4.115384615384615)
Win/lose count 8.0/10.0. Average score (-3.9642857142857144)
Win/lose count 13.0/10.0. Average score (-3.5)
Final score: -3.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
> ** Let's show that ** :

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
We have : 


\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\ =  r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\ =r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s',a_{0}=a'] \\ = r(s,a) +E_{(s',a')\sim p(.|s,a)} [\gamma E_{p^{\pi}}[\sum_{t=0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']]  \\ = E_{(s',a')\sim p(.|s,a)} [r(s,a) +\gamma E_{p^{\pi}}[\sum_{t=0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] \\ Q^{\pi}(s,a) =  E_{(s',a')\sim p(.|s,a)} [r(s,a) +\gamma Q^{\pi}(s',a') ]
\end{equation*}


***
> ** Let's show that ** :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
We know that : 
\begin{equation*}
\pi^{*}(s')=argmax_{a'} Q^* (s',a').
\end{equation*}

So :

\begin{equation*}
Q^{*}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]. \\ = E_{s'\sim p(.|s,a),a'\sim \pi^{*}(.|s')  }[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].\\Q^{*}(s,a) = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

***
> ** Let's show that ** :

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

We have : 

$$Q^{*}(s,a) = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].$$ 

which means that $Q^{*}$  verifies the following equality :

\begin{equation*}
E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q(s',a') - Q(s,a)] = 0
\end{equation*}

That means that  the optimal solution is reached by minimizing :

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]

    def random_access(self):
        return random.choice(self.memory)
        

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            # Save the current state
            input_states[i] = s_
            
            # Modifying/updating the Q function
            
            
            if game_over_:
              #Only the reward is taken because ne next state 
              target_q[i,a_] = r_
               
               
            else:
              target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
      
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.01,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        # Flattening
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        # Using a single hidden layer 
        model.add(Dense(20,activation ='relu')) 
        # Fully connected ouptput layer 
        model.add(Dense(4)) # 4 possible actions
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))
#HTML(display_videos('fc_train10.mp4'))

Epoch 000/045 | Loss 0.0273 | Win/lose count 3.5/10.0 (-6.5)
Epoch 001/045 | Loss 0.0125 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/045 | Loss 0.0436 | Win/lose count 7.0/9.0 (-2.0)
Epoch 003/045 | Loss 0.0083 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/045 | Loss 0.0067 | Win/lose count 3.0/6.0 (-3.0)
Epoch 005/045 | Loss 0.0152 | Win/lose count 3.5/3.0 (0.5)
Epoch 006/045 | Loss 0.0066 | Win/lose count 6.5/4.0 (2.5)
Epoch 007/045 | Loss 0.0044 | Win/lose count 6.5/3.0 (3.5)
Epoch 008/045 | Loss 0.0080 | Win/lose count 5.5/0 (5.5)
Epoch 009/045 | Loss 0.0079 | Win/lose count 3.5/1.0 (2.5)
Epoch 010/045 | Loss 0.0110 | Win/lose count 5.0/4.0 (1.0)
Epoch 011/045 | Loss 0.0104 | Win/lose count 3.0/1.0 (2.0)
Epoch 012/045 | Loss 0.0007 | Win/lose count 4.5/4.0 (0.5)
Epoch 013/045 | Loss 0.0092 | Win/lose count 7.5/4.0 (3.5)
Epoch 014/045 | Loss 0.0043 | Win/lose count 3.0/4.0 (-1.0)
Epoch 015/045 | Loss 0.0072 | Win/lose count 2.5/3.0 (-0.5)
Epoch 016/045 | Loss 0.0486 | Win/lose count 3.0/2.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        
        # A conv layer of kernel size 3 and 30 filters
        model.add(Conv2D(30,(3,3),input_shape=(5,5,self.n_state,),activation='relu'))
        
        # A conv layer of kernel size 3 and 20 filters
        model.add(Conv2D(20,(3,3),activation='relu'))
        
        # Flattening  the output
        model.add(Flatten())
        
        # Flattening and feeding the output to a 4 units fully connected layer
        model.add(Dense(4))
    
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
#HTML(display_videos('cnn_train0.mp4'))

Epoch 000/045 | Loss 0.0271 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/045 | Loss 0.0022 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/045 | Loss 0.0091 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/045 | Loss 0.0052 | Win/lose count 3.5/7.0 (-3.5)
Epoch 004/045 | Loss 0.0101 | Win/lose count 3.5/5.0 (-1.5)
Epoch 005/045 | Loss 0.0073 | Win/lose count 5.5/5.0 (0.5)
Epoch 006/045 | Loss 0.0903 | Win/lose count 6.0/3.0 (3.0)
Epoch 007/045 | Loss 0.0057 | Win/lose count 5.5/3.0 (2.5)
Epoch 008/045 | Loss 0.0022 | Win/lose count 2.5/0 (2.5)
Epoch 009/045 | Loss 0.0040 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/045 | Loss 0.0024 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/045 | Loss 0.0027 | Win/lose count 3.5/1.0 (2.5)
Epoch 012/045 | Loss 0.0030 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/045 | Loss 0.0049 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/045 | Loss 0.0012 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/045 | Loss 0.0011 | Win/lose count 5.5/1.0 (4.5)
Epoch 016/045 | Loss 0.0516 | Win/lose count 9.0/2.0 (7

In [0]:
HTML(display_videos('cnn_train10.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 9.5/5.0. Average score (4.5)
Win/lose count 15.5/6.0. Average score (7.0)
Win/lose count 21.0/4.0. Average score (10.333333333333334)
Win/lose count 17.5/3.0. Average score (11.375)
Win/lose count 14.0/9.0. Average score (10.1)
Win/lose count 19.5/3.0. Average score (11.166666666666666)
Win/lose count 14.0/6.0. Average score (10.714285714285714)
Win/lose count 21.0/5.0. Average score (11.375)
Win/lose count 12.0/5.0. Average score (10.88888888888889)
Win/lose count 11.0/5.0. Average score (10.4)
Win/lose count 8.0/3.0. Average score (9.909090909090908)
Win/lose count 12.0/2.0. Average score (9.916666666666666)
Win/lose count 4.5/0. Average score (9.5)
Win/lose count 4.5/1.0. Average score (9.071428571428571)
Win/lose count 20.0/5.0. Average score (9.466666666666667)
Final score: 9.466666666666667
Test of the FC
Win/lose count 7.5/5.0. Average score (2.5)
Win/lose count 12.0/12.0. Average score (1.25)
Win/lose count 9.0/7.0. Average score (1.5)
Win/lose co

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

> ** Observation and comment** : The issues that we observe are that the agent get stuck in an area of the grid if he is not "surrounded" by a positive cell (not in his visibility). And this case is specially observed fo the fully connected architecture.

> ** Effect of temperature** Increasing the temperature means increasing the number of positive cells which means that the agent gets to explore the map easily than a smaller value because he will be surrounded by more positive cells, which means a better final verage score.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay = 0.1,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    # Starting the value of epsilone
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        
        agent.set_epsilon(agent.epsilon*(1-decay))
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        # apply a malus if thethe agent visit an already visited cell 
        self.malus_position = np.zeros((grid_size,grid_size))
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

            
        
        self.t = self.t + 1
        
        self.malus_position[self.x, self.y] = 0.1
        
        r = 0
        if train:
            r = -self.malus_position[self.x, self.y]
            
        reward = self.board[self.x, self.y] + r 
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0

        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2) 

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
      

In [0]:
# Training
# Parameters
size = 13
T=200
temperature=0.3
epochs_train=100
epochs_test=15


env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')


Epoch 000/100 | Loss 0.0144 | Win/lose count 9.600000000000003/24.700000000000088 (-15.100000000000085)
Epoch 001/100 | Loss 0.0292 | Win/lose count 5.2/26.800000000000093 (-21.600000000000094)
Epoch 002/100 | Loss 0.0059 | Win/lose count 2.8/29.400000000000123 (-26.600000000000122)
Epoch 003/100 | Loss 0.0083 | Win/lose count 10.000000000000004/22.600000000000065 (-12.600000000000062)
Epoch 004/100 | Loss 0.0154 | Win/lose count 2.0/21.60000000000004 (-19.60000000000004)
Epoch 005/100 | Loss 0.0054 | Win/lose count 5.6000000000000005/28.700000000000117 (-23.100000000000115)
Epoch 006/100 | Loss 0.0060 | Win/lose count 5.2/24.80000000000005 (-19.60000000000005)
Epoch 007/100 | Loss 0.0104 | Win/lose count 6.400000000000001/22.50000000000005 (-16.100000000000048)
Epoch 008/100 | Loss 0.0081 | Win/lose count 7.200000000000002/21.300000000000043 (-14.10000000000004)
Epoch 009/100 | Loss 0.0171 | Win/lose count 16.800000000000004/21.900000000000055 (-5.100000000000051)
Epoch 010/100 | Loss

In [0]:
HTML(display_videos('cnn_train_explore10.mp4'))

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore14.mp4'))

Win/lose count 16.0/2.0. Average score (14.0)
Win/lose count 8.5/0. Average score (11.25)
Win/lose count 15.0/1.0. Average score (12.166666666666666)
Win/lose count 12.0/0. Average score (12.125)
Win/lose count 17.5/2.0. Average score (12.8)
Win/lose count 1.5/0. Average score (10.916666666666666)
Win/lose count 9.5/2.0. Average score (10.428571428571429)
Win/lose count 16.5/0. Average score (11.1875)
Win/lose count 23.5/2.0. Average score (12.333333333333334)
Win/lose count 15.5/3.0. Average score (12.35)
Win/lose count 13.0/2.0. Average score (12.227272727272727)
Win/lose count 10.5/2.0. Average score (11.916666666666666)
Win/lose count 20.5/3.0. Average score (12.346153846153847)
Win/lose count 20.0/3.0. Average score (12.678571428571429)
Win/lose count 6.5/1.0. Average score (12.2)
Final score: 12.2


Indeed the agent explores more areas in the map and we have a good final score. which means that the epsilone greedy strategy works well with this kind of problems.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***